In [54]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
# LightGBM Model
import sys
from pathlib import Path

# add parent directory to Python path
parent_dir = Path.cwd().parent
sys.path.insert(0, str(parent_dir))

In [59]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import src.data as data 
import src.features as features
import importlib
import src.metrics as metrics 

#Reload all the modules to make sure the changes to the dependency files are always live.
# importlib.reload(data)
# importlib.reload(features)

from sklearn.metrics import roc_auc_score, log_loss

# --------------------------
# 1) Load Criteo train.txt (no header)
# label + 13 numerical (I1..I13) + 26 categorical (C1..C26)
# --------------------------

nrows = 5000000           # <-- set None for full file (can be huge)
seeds = [11]
models = []

for seed in seeds:
    (num_features, 
     cat_features, 
     columns, 
     X_train_raw, 
     X_val_raw, 
     X_test_raw, 
     y_train, 
     y_val, 
     y_test) = data.load_data(
        data_path = "../data/criteo/train.txt", data_size = nrows, train_eval_random_state = seed)
    
    # --------------------------
    # 2) Label-encode categoricals (fit on TRAIN only)
    #    Unseen categories in val -> -1
    # --------------------------
    # encoders = {}
    # for c in cat_features:
    #     uniq = X_train[c].unique()
    #     encoders[c] = {v: i for i, v in enumerate(uniq)}
    #     X_train[c] = X_train[c].map(encoders[c]).astype(np.int32)
    #     X_val[c] = X_val[c].map(encoders[c]).fillna(-1).astype(np.int32)


    # 3) Add numeric engineered features
    X_train_raw, X_val_raw, X_test_raw, num_eng_cols = features.add_num_log_and_missing(
        X_train_raw, X_val_raw, X_test_raw, num_features
    )
    
    # 4.1) Add categorical frequency features (uses TRAIN distribution only)
    X_train_raw, X_val_raw, X_test_raw, freq_cols = features.add_cat_frequency(
        X_train_raw, X_val_raw, X_test_raw, cat_features
    )

    # 4.2) Add categorical log frequency features (uses TRAIN distribution only)
    X_train_raw, X_val_raw, X_test_raw, freq_cols = features.add_log_freq(
        X_train_raw, X_val_raw, X_test_raw, cat_features
    )

    # 5) Fill missing values.
    data.fill_missing_values(X_train_raw, X_val_raw, X_test_raw, num_features, cat_features)
    X_train = X_train_raw
    X_val = X_val_raw
    X_test = X_test_raw

    # 6) Convert cat_features to hashes
    features.convert_cat_to_hash(X_train, cat_features)
    features.convert_cat_to_hash(X_val, cat_features)

    # 7) Feature crossing on already hashed features
    X_train, X_val, X_test, I11_bin_col, I11_edges = features.add_quantile_bin(X_train, X_val, X_test, "I11", n_bins=64)
    HASH_BINS = 1 << 20  # 1,048,576
    
    # C10 × C17
    X_train, X_val, X_test, cross1 = features.add_hashed_cross(
        X_train, X_val, X_test,
        "C10", "C17",
        new_name="C10xC17",
        num_bins=HASH_BINS
    )
    
    # C10 × I11_bin
    X_train, X_val, X_test, cross2 = features.add_hashed_cross(
        X_train, X_val, X_test,
        "C10", I11_bin_col,
        new_name="C10xI11bin",
        num_bins=HASH_BINS
    )
    
    # --------------------------
    # 8) LightGBM datasets
    # --------------------------
    new_cat_cols = [cross1, cross2, I11_bin_col]     # treat bins as categorical
    new_num_cols = freq_cols                      # numeric
    
    feature_cols = list(X_train.columns)  # or explicitly build: num_eng_cols + cat_cols + freq_cols + logfreq_cols + [I11_bin_col, cross1, cross2]
    all_cat_features = cat_features + new_cat_cols

    dtrain = lgb.Dataset(X_train, label=y_train, categorical_feature=all_cat_features, free_raw_data=False)
    dval = lgb.Dataset(X_val, label=y_val, categorical_feature=all_cat_features, reference=dtrain, free_raw_data=False)
    
    # --------------------------
    # 9) Minimal params for CTR
    # --------------------------
    params = {
        "objective": "binary",
        "metric": ["auc", "binary_logloss"],
        "learning_rate": 0.03,
        "num_leaves": 255,
        "max_depth": 12,
        "min_data_in_leaf": 300,
        "feature_fraction": 0.8,
        "bagging_fraction": 0.8,
        "bagging_freq": 1,
        "lambda_l2": 10,
        "verbosity": -1,
        "seed": 42,
    }

    # --------------------------
    # 10) Train with early stopping
    # --------------------------
    model = lgb.train(
        params,
        dtrain,
        num_boost_round=2000,
        valid_sets=[dval],
        valid_names=["val"],
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=50),
        ],
    )
    models.append(model)

    # --------------------------
    # 11) Evaluate
    # --------------------------
    p_val = model.predict(X_val, num_iteration=model.best_iteration)
    
    print(metrics.metrics_basic(y_val, p_val))
    
    ece, ece_table = metrics.expected_calibration_error(y_val, p_val, n_bins=15)
    print("ECE:", ece)

    print(metrics.topk_lift(y_val, p_val, k=0.01))   # top 1%
    print(metrics.topk_lift(y_val, p_val, k=0.05))   # top 5

42


/Users/Riff/ctr-env/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/Riff/ctr-env/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/Riff/ctr-env/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/Riff/ctr-env/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/Riff/ctr-env/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/Riff/ctr-env/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWa

[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Met categoric

In [ ]:
with pd.option_context('display.max_columns', None):
    print(X_train.head(10))

In [61]:
# Run the model on test set.
#print(X_test.head(1))
features.convert_cat_to_hash(X_test, cat_features)
#print(X_test.head(1))

p_test = model.predict(X_test, num_iteration=model.best_iteration)
    
print(metrics.metrics_basic(y_test, p_test))

ece, ece_table = metrics.expected_calibration_error(y_test, p_test, n_bins=15)
print("ECE:", ece)

print(metrics.topk_lift(y_test, p_test, k=0.01))   # top 1%
print(metrics.topk_lift(y_test, p_test, k=0.05))   # top 5%

{'auc': 0.7946398988802232, 'logloss': 0.4517447045191126, 'pr_auc': 0.5831389395321789, 'brier': 0.14628356984096172, 'ctr_mean_true': 0.25125, 'ctr_mean_pred': 0.24875493089817768}
ECE: 0.00886980180107436
{'k': 0.01, 'n_top': 5000, 'ctr_top': 0.8912, 'ctr_all': 0.25125, 'lift': 3.547064676616902}
{'k': 0.05, 'n_top': 25000, 'ctr_top': 0.77744, 'ctr_all': 0.25125, 'lift': 3.0942885572139183}


In [89]:
# Save the last model to disk
model.save_model("../models/lgb/lgb_ctr_model.txt")


In [40]:
aucs = []
loglosses = []
briers = []
eces = []

for model in models:
    p_test = model.predict(X_test, num_iteration=model.best_iteration)
    
    mets = metrics.metrics_basic(y_test, p_test)
    aucs.append(mets['auc'])
    loglosses.append(mets['logloss'])
    briers.append(mets['brier'])
    
    ece, ece_table = metrics.expected_calibration_error(y_test, p_test, n_bins=15)
    eces.append(ece)

metrics.print_mean_std('AUC', aucs)
metrics.print_mean_std('loglosses',loglosses)
metrics.print_mean_std('briers',briers)
metrics.print_mean_std('eces',eces)

AUC     : 0.7962 ± nan
loglosses: 0.4501 ± nan
briers  : 0.1457 ± nan
eces    : 0.0039 ± nan


/Users/Riff/ctr-env/lib/python3.9/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/Riff/ctr-env/lib/python3.9/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


# Tuning history

## Tune tree model params 
"learning_rate", "num_leaves", "max_depth", "min_data_in_leaf", "lambda_l2"
and got the decent result below.

## Tune features

### Label-encode categorical features seed = 42:
Test AUC    : 0.7182091248139506
Test LogLoss: 0.5234532609680653


### hashing categorical features seed = 42, <span style="color:red;">Big Improvement!</span> 

Test AUC    : 0.7883122347769331
Test LogLoss: 0.4587729242271498

seed = 19
Test AUC    : 0.7874018158198559
Test LogLoss: 0.4593220177816222

seed = 10
Test AUC    : 0.7877994036835043
Test LogLoss: 0.45899872089590293
Best iter  : 620


### nrows = 2000000 Train eval test split 0.8, 0.1, 0.1 seed = 10:
params = {
    "objective": "binary",
    "metric": ["auc", "binary_logloss"],
    "learning_rate": 0.03,
    "num_leaves": 255,
    "max_depth": 12,
    "min_data_in_leaf": 300,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 1,
    "lambda_l2": 10,
    "verbosity": -1,
    "seed": 42,
}

Test
{'auc': 0.7958963633226189, 'logloss': 0.450386281860432, 'pr_auc': 0.5856542748459541, 'brier': 0.14579051407211444, 'ctr_mean_true': 0.25125, 'ctr_mean_pred': 0.25023126129109174}
ECE: 0.00551766077415308
{'k': 0.01, 'n_top': 5000, 'ctr_top': 0.898, 'ctr_all': 0.25125, 'lift': 3.574129353233817}
{'k': 0.05, 'n_top': 25000, 'ctr_top': 0.7814, 'ctr_all': 0.25125, 'lift': 3.110049751243769}

### Same LightGBM params as above, nrows = 5000000 Train eval test split 0.8, 0.1, 0.1 seed = 11, 22, 33, get the mean and std for the metrics:

AUC : 0.7961 ± 0.0002 
loglosses: 0.4502 ± 0.0001 
briers : 0.1457 ± 0.0000 
eces : 0.0038 ± 0.0006

### With log1p, freq and missing features added:

AUC     : 0.7961 ± 0.0001
loglosses: 0.4501 ± 0.0001
briers  : 0.1457 ± 0.0000
eces    : 0.0037 ± 0.0005

{'auc': 0.7962079710668694, 'logloss': 0.4500324640233981, 'pr_auc': 0.5861662260191162, 'brier': 0.14565938497056644, 'ctr_mean_true': 0.25125, 'ctr_mean_pred': 0.2496694684545241}
ECE: 0.00362046996990802
{'k': 0.01, 'n_top': 5000, 'ctr_top': 0.8926, 'ctr_all': 0.25125, 'lift': 3.5526368159203843}
{'k': 0.05, 'n_top': 25000, 'ctr_top': 0.78192, 'ctr_all': 0.25125, 'lift': 3.1121194029850625}

### There was a bug above, missing features was called after fillna(), this is not intended, we fixed it! 

{'auc': 0.7964011491019027, 'logloss': 0.4498488910681045, 'pr_auc': 0.5866849981967813, 'brier': 0.14560254260769245, 'ctr_mean_true': 0.25125, 'ctr_mean_pred': 0.25001718486419217}
ECE: 0.004156878282011214
{'k': 0.01, 'n_top': 5000, 'ctr_top': 0.8986, 'ctr_all': 0.25125, 'lift': 3.5765174129353094}
{'k': 0.05, 'n_top': 25000, 'ctr_top': 0.78296, 'ctr_all': 0.25125, 'lift': 3.1162587064676495}

### Features above included along with feature crossing and log_freq, some dgradation  in performance observed:

{'auc': 0.7946398988802232, 'logloss': 0.4517447045191126, 'pr_auc': 0.5831389395321789, 'brier': 0.14628356984096172, 'ctr_mean_true': 0.25125, 'ctr_mean_pred': 0.24875493089817768}
ECE: 0.00886980180107436
{'k': 0.01, 'n_top': 5000, 'ctr_top': 0.8912, 'ctr_all': 0.25125, 'lift': 3.547064676616902}
{'k': 0.05, 'n_top': 25000, 'ctr_top': 0.77744, 'ctr_all': 0.25125, 'lift': 3.0942885572139183}

# Shap analysis

In [82]:
import shap
import matplotlib.pyplot as plt

explainer = shap.TreeExplainer(model)
X_val_samples = X_val.sample(n=50*1000, replace=False, random_state=42)
shap_values = explainer.shap_values(X_val_samples)

/Users/Riff/ctr-env/lib/python3.9/site-packages/shap/explainers/_tree.py:586: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(


In [83]:
# Global summary
shap.summary_plot(shap_values, X_val_samples, show=False)
plt.tight_layout()
plt.savefig("../images/shap_summary.png", dpi=200, bbox_inches="tight")
plt.close()

In [84]:
# pick one row to explain (from val or test)
i = 15
x = X_val_samples.iloc[[i]]   # keep as DataFrame

# SHAP contributions (log-odds space)
contrib = model.predict(
    x,
    pred_contrib=True,
    num_iteration=model.best_iteration
)

shap_values_single = contrib[0, :-1]   # per-feature contributions
base_value_single  = contrib[0, -0]    # expected value (log-odds)

In [85]:
exp = shap.Explanation(
    values=shap_values_single,
    base_values=base_value_single,
    data=x.values[0],
    feature_names=X_val.columns
)

shap.waterfall_plot(exp, show=False)
plt.tight_layout()
plt.savefig("../images/shap_waterfall15.png", dpi=200, bbox_inches="tight")
plt.close()


# SHAP analysis report

## SHAP summary
<div align="center">
  <img src="../images/shap_summary.png" width="500">
</div>

### Top drivers are:

- Numericals: I11, I6, I7
- Categoricals: C14, C15, C17, C23, C13, C7

### cross feature: C10xC17

It appears very low in the ranking, near the bottom.

That tells us:
- The interaction exists
- But it is weak relative to base features
- It is not a dominant structural interaction
- This matches the AUC drop — the model is not gaining meaningful signal from it.

### Log-frequency features

They are not among top global drivers.

This is important:
The model is not strongly using log(freq) features.
So either:
- Raw categorical IDs already capture the necessary signal
- Frequency abstraction doesn’t add much at 5M scale
- Or frequency features are too correlated with base IDs

### Visual observation: slightly more noise spread

The SHAP horizontal spread looks slightly wider for lower-ranked features.
This suggests:
- Added features increased model flexibility
- But not necessarily improved generalization

That’s consistent with:
- Slight AUC drop
- Worse ECE

## Waterfall Plot (Single Example)

<div align="center">
  <img src="../images/shap_waterfall15.png" width="500">
</div>


### This is a high-CTR example (f(x) = 0.759).

Baseline:
E[f(X)] = -0.019

- This baseline changed compared to earlier model — that’s a signal.

### Dominant positive driver: C13 = 4916 (+0.49)

Very strong ID effect.

Interpretation:

- Raw categorical still dominates.

- Cross feature did not replace this.

### Negative drivers: I11, C14, I6

Again:

- Numericals matter strongly.

- Cross feature not dominating.

### Cross feature effect?

C10xC17 does not appear in top local contributors.

That confirms:

- The cross is weak for this sample.

## Big Picture Takeaways
### Your baseline model was near the signal ceiling.

At 5M rows, LightGBM already extracted most of the signal.
### Cross features are not strong enough in this dataset.

Not every intuitive cross helps.
### Frequency abstraction does not dominate.

### This suggests:
- Either data already large enough
- Or frequency signal overlaps with raw ID splits

### SHAP validates metrics.

- This is important:
  - The visual interpretation matches the numeric regression.

- That means:
  - The experimental workflow is correct.